In [31]:
import backtrader as bt
import backtrader.analyzers as btanalyzers
import matplotlib
from datetime import datetime
 
class MaCrossStrategy(bt.Strategy):
    params = (
        ('fast_length', 5),
        ('slow_length', 25),
        ('rsi_upper', 70),
        ('rsi_lower', 30)
    )
     
    def __init__(self):
        self.crossovers = []
        self.rsi = []
         
        for d in self.datas:
            ma_fast = bt.ind.SMA(d, period = self.params.fast_length)
            ma_slow = bt.ind.SMA(d, period = self.params.slow_length)
            self.crossovers.append(bt.ind.CrossOver(ma_fast, ma_slow))
            
            self.rsi.append(bt.ind.RSI_Cutler())
 
    def next(self):
        for i, d in enumerate(self.datas):
            if self.crossovers[i] > 0: 
                self.buy(data=d, size=10)

            if self.rsi[i] < self.params.rsi_lower and self.getposition(d).size < 1000:
                self.buy(data=d, size=20)
            elif self.rsi[i] > self.params.rsi_upper and self.getposition(d).size > 0:
                self.sell(data=d, size=20)

cerebro = bt.Cerebro()
 
stocks = ['AAPL', 'MSFT', 'AMZN', 'TSLA']
for s in stocks: 
    data = bt.feeds.YahooFinanceData(dataname = s, fromdate = datetime(2010, 1, 1), todate = datetime(2020, 1, 1))
    cerebro.adddata(data, name = s)
 
 
cerebro.addstrategy(MaCrossStrategy)
 
cerebro.broker.setcash(100000.0)

sizer_percent = 50
cerebro.addsizer(bt.sizers.PercentSizer, percents = sizer_percent)
 
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name = "sharpe")
cerebro.addanalyzer(btanalyzers.Returns,     _name = "returns")
cerebro.addanalyzer(btanalyzers.Transactions, _name = "trans")
 
back = cerebro.run()
 
final_portfolio_value = cerebro.broker.getvalue()
# positions = cerebro.broker.getposition(data)
avg_yearly_return = back[0].analyzers.returns.get_analysis()['rnorm100']
sharpe_ratio = back[0].analyzers.sharpe.get_analysis()['sharperatio']
num_trans = len(back[0].analyzers.trans.get_analysis())

In [32]:
print(f'Final Portfolio Value: {final_portfolio_value}')
print(f'Average Yearly Returns: {avg_yearly_return}')
print(f'Sharpe Ratio: {sharpe_ratio}')
print(f'Number of Transactions: {num_trans}')
print(f'Sizer Percent: {sizer_percent}')
# print(positions)

Final Portfolio Value: 234958.30000000022
Average Yearly Returns: 8.93264783024676
Sharpe Ratio: 0.8074879972599336
Number of Transactions: 552
Sizer Percent: 50
